<a href="https://colab.research.google.com/github/fjme95/calculo-optimizacion/blob/main/Semana%204/Stochastic_Gradient_Boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencias

In [ ]:
!pip install -U plotly

     |████████████████████████████████| 25.3 MB 6.0 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [ ]:
from time import time

import tensorflow as tf  # Para cargar el dataset. Nos evitamos descargar manualmente los 4 archivos del dataset y utilizar gzip y numpy.frombuffer para leer los datos

import numpy as np

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

from skimage.transform import resize

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

# Datos

Para hacer las pruebas, ocuparemos el dataset de MNIST que contiene imágenes de 29x29 de digitos escritos a mano. El dataset está dividido en un conjunto de entrenamiento con 60,000 imágenes y uno de prueba con 10,000. Cada uno con sus respectivas etiquetas.

Para evitar tiempos de entrenamiento prologandos, se hará lo siguiente:

1. El conjunto de entrenamiento se reducirá a 1000 imágenes.
2. La imágenes se reduciran de 28x28 a 14x14.

## Descarga y lectura de los datos

In [ ]:
(X_train, y_train), (X_test, y_test) =  tf.keras.datasets.mnist.load_data()
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


## Reducción del conjunto de entrenamiento

In [ ]:
X_train, _, y_train, _ = train_test_split(X_train, y_train, train_size = 1000, random_state = 10)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(1000, 28, 28) (1000,) (10000, 28, 28) (10000,)


## Redimensionamiento de las imágenes

In [ ]:
X_train = np.array([resize(img, (14,14)) for img in X_train])
X_test = np.array([resize(img, (14,14)) for img in X_test])
X_train.shape

(1000, 14, 14)

##Visualización de los datos

In [ ]:
label = None
n = 10 # Debe ser par

if label is None:
    labels = y_train[:n]
    images = X_train[:n]
else:
    labels = y_train[y_train == label][:n]
    images = X_train[y_train == label][:n]

fig = make_subplots(
    rows=2, cols=int(n/2),
    subplot_titles=labels.tolist())

for i in range(n):
    fig.add_trace(px.imshow(255-images[i], binary_string=True).data[0], row = int(i//(n/2)) + 1, col = int(i%(n/2)) + 1)

fig.show()

## Conjunto de entrenamiento "aplanado"

In [ ]:
X_train_flat = np.array([x.ravel() for x in X_train])
X_train_flat.shape

(1000, 196)

# Gradient Boosting

Primero se hará el entrenamiento con Gradient Boosting que por default tiene una tasa de aprendizaje (learning rate) de .1.

## Entrenamiento del modelo

In [ ]:
start = time()
n_estimators = 500
clf = GradientBoostingClassifier(n_estimators=n_estimators)
clf.fit(X_train_flat, y_train)
print(time() - start)

89.80414366722107


## Aplanamiento del conjunto de prueba

In [ ]:
X_test_flat = [x.ravel() for x in X_test]
clf_test_pred = clf.staged_decision_function(X_test_flat)

## Evaluación de test

La función ```staged_decision_function``` regresa un *generador* con las predicciones del modelo tomando cada vez más estimadores. Por ejemplo, si el estimador es el árbol de decisión, el primer elemento del generador es la predicción con el primer árbol del modelo, la segunda es la predicción con los primeros dos árboles, etc.

In [ ]:
test_deviance = np.zeros(n_estimators, dtype=np.float64)
for i, y_pred in enumerate(clf.staged_decision_function([x.ravel() for x in X_test])):
    test_deviance[i] = clf.loss_(y_test, y_pred)

## Visualización de la pérdida

In [ ]:
fig = px.line(test_deviance,
        title = 'Pérdida para Gradient Boosting', 
        labels={
            "index": "Número de árboles",
            "value": "Pérdida en el set de entrenamiento",
            'variable': ''
            }, 
        )
fig.update_traces(name='Sin regularización')

# Stochastic Gradient Boosting

Se hará lo mismo que en la sección anterior, utilizando valores distintos para el parámetro ```subsample``` de ```GradientBoostingClassifier```. Cuando éste es distinto de 1, obtenemos Stochastic Gradient Boosting.

## Comparación entre distintos valores de subsample

In [ ]:
times = []
# default 1
subsamples = [.3, .5, .7, .9, 1]

fig = go.Figure()

for subsample in subsamples:
    start = time()
    clf = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample)
    clf.fit(X_train_flat, y_train)
    times.append(time() - start)

    clf_test_pred = clf.staged_decision_function(X_test_flat)

    test_deviance = np.zeros(n_estimators, dtype=np.float64)
    for i, y_pred in enumerate(clf.staged_decision_function([x.ravel() for x in X_test])):
        test_deviance[i] = clf.loss_(y_test, y_pred)

    fig.add_trace(go.Scatter(y = test_deviance, name = f'subsample = {subsample}'))

fig.show()

## Comparación con distintos valores de learning_rate

In [ ]:
times = []
# default .1
lrs = [.001, .05, .1, .5, 1]

fig = go.Figure()

for lr in lrs:
    start = time()
    clf = GradientBoostingClassifier(n_estimators=n_estimators, subsample=.3, learning_rate=lr)
    clf.fit(X_train_flat, y_train)
    times.append(time() - start)

    clf_test_pred = clf.staged_decision_function(X_test_flat)

    test_deviance = np.zeros(n_estimators, dtype=np.float64)
    for i, y_pred in enumerate(clf.staged_decision_function([x.ravel() for x in X_test])):
        test_deviance[i] = clf.loss_(y_test, y_pred)

    fig.add_trace(go.Scatter(y = test_deviance, name = f'learning_rate = {lr}'))

fig.show()